In [ ]:
import requests
import csv
import tabulaimport pandas as pd
import re

#Reading Appeal document file
appeal_doc = pd.read_csv('appeal_document.csv')

#Reading Appeal file
appeal = pd.read_csv('appeal.csv')

#Filtering data for Flood
appeal_doc = appeal_doc[appeal_doc.name.str.lower().str.contains('final')]
appeal = appeal[appeal['dtype.name'] == 'Flood']
appeal_doc = appeal_doc[appeal_doc.appeal.isin(appeal.code.unique().tolist())]

# Function to extract tables from a PDF URL using Tabula
def extract_tables_from_pdf(pdf_url):
    try:
        # Download the PDF file
        response = requests.get(pdf_url)
        response.raise_for_status()

        # Save the PDF locally
        with open("temp.pdf", "wb") as f:
            f.write(response.content)

        # Extract tables from the PDF
        tables = tabula.read_pdf("temp.pdf", pages="all")

        return tables
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {pdf_url}: {str(e)}")
        return None
    except Exception as e:
        print(f"Error extracting tables from {pdf_url}: {str(e)}")
        return None
    finally:
        # Clean up
        response.close()
        os.remove("temp.pdf")

# Creating a CSV file to write data
with open("Tabula_Flood_output.csv", "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)

    # Loop through the PDF URLs and extract tables
    for pdf_url in appeal_doc.document_url.unique().tolist():
        tables = extract_tables_from_pdf(pdf_url)
        if tables:
            for table in tables:
                # Add the PDF URL as a new column to use as a mapping id
                table["document_url"] = pdf_url

                # Check if the table contains 'Total', 'Budget', and 'Expenditure' in any row
                if 'Total' in table.values and 'Budget' in table.values and 'Expenditure' in table.values:
                    # Write table rows
                    for _, row in table.iterrows():
                        csv_writer.writerow(row)




In [ ]:
# This part is to handle some issues in created output file

#Read output file
df = pd.read_csv('Tabula_Flood_output.csv')


def check_number_start_with_p_or_s(text):
    if text.startswith('p'):
        text = text[1:]  # Remove the 'p' prefix
    elif text.startswith('s'):
        text = text[1:]  # Remove the 's' prefix

    # Remove periods and single quotes between digits
    text = re.sub(r'(?<=\d)\.(\d)|\'(\d)', r'\1\2', text)

    return text


def extract_number_from_column1(text):
    # Split the text by whitespace
    words = text.split()

    # Initialize variables for the numbers
    num1 = None
    num2 = None

    # Iterate through the words in reverse order
    for word in reversed(words):
        # Check if the word is a number with or without commas
        if re.match(r'^-?[\d,]+$', word):
            if num1 is None:
                num1 = word
            else:
                num2 = word
                break  # We found the last two numbers

    # If no numbers are found, return text
    if num1 is None:
        return [text]

    # If one number is found, return text and the single number
    if num2 is None:
        text_without_number = ' '.join(words[:-1])  # Exclude the last word (the number)
        return [text_without_number, check_number_start_with_p_or_s(num1)]

    # If two numbers are found, return text, num1, and num2
    text_without_numbers = ' '.join(words[:-2])  # Exclude the last two words (the numbers)
    return [text_without_numbers, check_number_start_with_p_or_s(num1), check_number_start_with_p_or_s(num2)]


dfs = []

for i, row in df.iterrows():
    row_dict = {}
    row_dict['document_url'] = row['document_url']
    row_dict['Filename'] = row['Filename']
    k = extract_number_from_column1(row['Column1'])

    if len(k) == 3:
        row_dict['Item Name'] = k[0]
        row_dict['Budget'] = k[1]
        row_dict['ExpT1'] = k[2]
    elif len(k) == 2:
        row_dict['Item Name'] = k[0]
        row_dict['Budget'] = k[1]
    else:
        row_dict['Item Name'] = row['Column1']
        row_dict['Budget'] = check_number_start_with_p_or_s(str(row['Column2']))


    expT1_value = ''
    expT_value = ''
    var_value = ''
    misc_value = ''
    misc1_value = ''  # Initialize 'Misc1' value

    for col_num in range(3, 101):
        col_name = f'Column{col_num}'
        cell_value = row[col_name]


        if not pd.isna(cell_value):
            if len(k) != 3 and not expT1_value:
                expT1_value = check_number_start_with_p_or_s(cell_value)
            elif not expT_value:
                expT_value = check_number_start_with_p_or_s(cell_value)
            elif not var_value:
                var_value = check_number_start_with_p_or_s(cell_value)
            elif not misc_value:
                misc_value = check_number_start_with_p_or_s(cell_value)
            elif not misc1_value:  # Check for 'Misc1' value
                misc1_value = check_number_start_with_p_or_s(cell_value)
                break
    if len(k) != 3:
      row_dict['ExpT1'] = expT1_value
    row_dict['ExpT'] = expT_value
    row_dict['Var'] = var_value
    row_dict['Misc'] = misc_value
    row_dict['Misc1'] = misc1_value  # Add 'Misc1' to the row dictionary

    dfs.append(row_dict)

finaldf = pd.DataFrame(dfs)


In [ ]:
def process_row(row):
    # Split Budget into two numbers and update
    budget_parts = row['Budget'].split()
    if len(budget_parts) == 2:
        row['Misc1'] = row['Misc']
        row['Misc'] = row['Var']
        row['Var'] = row['ExpT']
        row['ExpT'] = row['ExpT1']
        row['Budget'], row['ExpT1'] = budget_parts[0], budget_parts[1]

    # Split ExpT1 into two numbers and update
    expt1_parts = row['ExpT1'].split()
    if len(expt1_parts) == 2:
        row['Misc1'] = row['Misc']
        row['Misc'] = row['Var']
        row['ExpT1'], row['Var'] = expt1_parts[0], expt1_parts[1]

    Split ExpT into two numbers and update
    expt_parts = row['ExpT'].split()
    if len(expt_parts) == 2:
        row['Misc1'] = row['Misc']
        row['Misc'] = row['Var']
        row['ExpT'], row['Var'] = expt_parts[0], expt_parts[1]


    return row

In [ ]:
finaldf = finaldf.apply(process_row, axis=1)

#Save the output
finaldf.to_csv('Flood_tabula_processd_file.csv',index=False)